# Text to Schematics : A Shap-e adapter

The goal of this reporsitory is to provide a adatation of [OpenAI shap-e](https://github.com/openai/shap-e) to generate minecraft schematics 

This repo also use the [Faithful repo](https://github.com/Faithful-Resource-Pack/Faithful-Java-32x) to generate the block list you can dapt this block list in the dedicate cell



---

## How to use it

- Run the celle 1 to 7 to intilise the repo
- write your text in the cell 8 
- Run the cell 8 to the end
- get the result in Out folder
- dowload the schematics
- Add the schematics in your server ```/plugin/WorldEdit/Schematics```
- Load the schematics  ```//shcem load my_schem```

In [ ]:
#@title Inisilisation


!git clone https://github.com/openai/shap-e.git

#Clone of orignal project
%cd shap-e
!pip install -e .
!pip install open3d==0.16.0
!pip install mcschematic
!pip install -q gradio

#Clone the Faithful
!wget https://github.com/Faithful-Resource-Pack/Faithful-Java-32x/releases/download/july-2022-release/Faithful.32x.-.1.19.zip
!unzip Faithful.32x.-.1.19.zip

#Create directory
!mkdir ../out 

import os
import cv2
import numpy as np

import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget

from functools import lru_cache

@lru_cache(maxsize=512)
def close_color(r, g, b):
    target_color = np.array([r*255, g*255, b*255])
    min_dist = float('inf')
    closest_block = None
    for block_name, avg_color in block_dict.items():
        dist = np.linalg.norm(target_color - np.array(avg_color))
        if dist < min_dist:
            min_dist = dist
            closest_block = block_name
    return closest_block

directory = "/content/shap-e/assets/minecraft/textures/block/" # replace with the path to your directory
block_dict = {}


excluded_words = ["front", "top", "bottom", "side","powder","glazed"]
white_list = ["wool","concrete","terracotta"]

for filename in os.listdir(directory):
    if filename.endswith(".png"):
        # extract the block name from the filename
        block_name = os.path.splitext(filename)[0]
        find = False
        for white in white_list:
          if white in block_name:
            find = True
            break
        
        if not find :
          continue

        # remove excluded words from block name
        for word in excluded_words:
            block_name = block_name.replace(word, "")
        block_name = block_name.strip("_").replace("__","_") # remove any trailing underscores
        # read the image and convert to RGB
        img = cv2.imread(os.path.join(directory, filename))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # calculate the average color
        avg_color = img.mean(axis=0).mean(axis=0)
        # save the average color in the dictionary
        block_dict[block_name] = tuple(avg_color.astype(int))


print(block_dict)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
xm = load_model('transmitter', device=device)
model = load_model('text300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [ ]:

from shap_e.util.notebooks import decode_latent_mesh
import numpy as np
import open3d as o3d
import uuid
import mcschematic
import logging

def generate_schematics(prompt):
  logging.warning(f'strart {prompt}')
  rdm = uuid.uuid1()
  batch_size = 4
  guidance_scale = 15.0

  latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(texts=[prompt] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
  )


  
  for i, latent in enumerate(latents):
    with open(f"out_{rdm}.ply", 'wb') as f:
      decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)
      if i == 3:
        break

  pcd = o3d.io.read_point_cloud(f'out_{rdm}.ply')
  pcd.scale(1 / np.max(pcd.get_max_bound() - pcd.get_min_bound()),
            center=pcd.get_center())
  voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd,
                                                              voxel_size=0.01)
  voxels = voxel_grid.get_voxels()

  schem = mcschematic.MCSchematic()

  for i in range(len(voxels)):
      colorr = voxels[i].color
      index = voxels[i].grid_index
      #print(index)
      block = close_color(colorr[0],colorr[1],colorr[2])
      #rotation when sqve the file
      schem.setBlock(  (index[0], index[2], index[1]), "minecraft:"+block )

  name = prompt.replace(" ","_")
  schem.save(  "../out", f"{name}_{rdm}", mcschematic.Version.JE_1_19_2)
  print("end")
  return  f"/content/out/{name}_{rdm}.schem"

In [ ]:
import gradio

gradio.Interface(generate_schematics, "text", "file").queue().launch(share=True,debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://a5d49cfb6e8d642a77.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


  0%|          | 0/64 [00:00<?, ?it/s]

/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


end


  0%|          | 0/64 [00:00<?, ?it/s]

...

end


  0%|          | 0/64 [00:00<?, ?it/s]

end


  0%|          | 0/64 [00:00<?, ?it/s]

end
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://a5d49cfb6e8d642a77.gradio.live
